<a href="https://colab.research.google.com/github/ritigupta07/Image-translation-Deep-Learning/blob/master/ML_MODEL_with_dataset_as_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Image translation using context and translated text with google API
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import re
df = pd.read_csv('/content/drive/My Drive/dataSet4.csv')
df.insert(5,'input_and_context','EMPTY')
for index, row in df.iterrows():
  row['input_and_context'] = str(row['english']).lower()
  #row['input_and_context'] = str(row['english']).lower() + ' <START> ' + str(row['context']).lower()
    #row['input_and_context'] = re.sub(r'[^\w]', ' ', row['english'].lower()) + ' <START> ' + re.sub(r'[^\w]', ' ', row['context'].lower())

In [0]:
import os


def load_data(path):
    """
    Load dataset
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data.split('\n')

In [0]:
import numpy as np
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical


def _test_model(model, input_shape, output_sequence_length, hindi_vocab_size):
    if isinstance(model, Sequential):
        model = model.model

    assert model.input_shape == (None, *input_shape[1:]),\
        'Wrong input shape. Found input shape {} using parameter input_shape={}'.format(model.input_shape, input_shape)

    assert len(model.loss_functions) > 0,\
        'No loss function set.  Apply the `compile` function to the model.'


def test_tokenize(tokenize):
    sentences = [
        'The quick brown fox jumps over the lazy dog .',
        'By Jove , my quick study of lexicography won a prize .',
        'This is a short sentence .']
    tokenized_sentences, tokenizer = tokenize(sentences)
    assert tokenized_sentences == tokenizer.texts_to_sequences(sentences),\
        'Tokenizer returned and doesn\'t generate the same sentences as the tokenized sentences returned. '


def test_pad(pad):
    tokens = [
        [i for i in range(4)],
        [i for i in range(6)],
        [i for i in range(3)]]
    padded_tokens = pad(tokens)
    padding_id = padded_tokens[0][-1]
    true_padded_tokens = np.array([
        [i for i in range(4)] + [padding_id]*2,
        [i for i in range(6)],
        [i for i in range(3)] + [padding_id]*3])
    assert isinstance(padded_tokens, np.ndarray),\
        'Pad returned the wrong type.  Found {} type, expected numpy array type.'
    assert np.all(padded_tokens == true_padded_tokens), 'Pad returned the wrong results.'

    padded_tokens_using_length = pad(tokens, 9)
    assert np.all(padded_tokens_using_length == np.concatenate((true_padded_tokens, np.full((3, 3), padding_id)), axis=1)),\
        'Using length argument return incorrect results'


def test_simple_model(simple_model):
    input_shape = (137861, 21, 1)
    output_sequence_length = 21
    english_vocab_size = 199
    hindi_vocab_size = 344

    model = simple_model(input_shape, output_sequence_length, english_vocab_size, hindi_vocab_size)
    _test_model(model, input_shape, output_sequence_length, hindi_vocab_size)


def test_embed_model(embed_model):
    input_shape = (137861, 21)
    output_sequence_length = 21
    english_vocab_size = 199
    hindi_vocab_size = 344

    model = embed_model(input_shape, output_sequence_length, english_vocab_size, hindi_vocab_size)
    _test_model(model, input_shape, output_sequence_length, hindi_vocab_size)


def test_encdec_model(encdec_model):
    input_shape = (137861, 15, 1)
    output_sequence_length = 21
    english_vocab_size = 199
    hindi_vocab_size = 344

    model = encdec_model(input_shape, output_sequence_length, english_vocab_size, hindi_vocab_size)
    _test_model(model, input_shape, output_sequence_length, hindi_vocab_size)


def test_bd_model(bd_model):
    input_shape = (137861, 21, 1)
    output_sequence_length = 21
    english_vocab_size = 199
    hindi_vocab_size = 344

    model = bd_model(input_shape, output_sequence_length, english_vocab_size, hindi_vocab_size)
    _test_model(model, input_shape, output_sequence_length, hindi_vocab_size)


def test_model_final(model_final):
    input_shape = (137861, 15)
    output_sequence_length = 21
    english_vocab_size = 199
    hindi_vocab_size = 344

    model = model_final(input_shape, output_sequence_length, english_vocab_size, hindi_vocab_size)
    _test_model(model, input_shape, output_sequence_length, hindi_vocab_size)

In [0]:
import collections

import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, Dropout, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.losses import sparse_categorical_crossentropy

In [0]:
# Load English data
#english_sentences = load_data('/content/drive/My Drive/data/small_vocab_en_gold')
# Load French data
#french_sentences = load_data('/content/drive/My Drive/data/small_vocab_fr_gold')
english_sentences = df['input_and_context']
hindi_sentences = df['hindi']
print('Dataset Loaded')

Dataset Loaded


In [0]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_hi Line {}:  {}'.format(sample_i + 1, hindi_sentences[sample_i]))

small_vocab_en Line 1:  st bernard's lily (anthericum liliago)
bermuda buttereup (oxalis pes-caprae)
oleanber (nerium oleander)
searlet pimpernel (anagallis arvensis)
verbascum (verbascum snuatuu)
lantana (lantana camara)
common mallow (malva 3ylvestris) spanish oyster (scolymus hispanicum)
stork's bill (erodium malacoides)
dindweed (convolvulus arvensis)
blue gem (hebes x franciscana)
calla lily (zantedeschia aethiopica)
a poster with flowers or clusters of flowers produced by
twelve species of flowering plants from different families.

small_vocab_hi Line 1:  St Bernarp's Lily (Anthericum liliago)
Bermuda Buttereup (Oxalis pes-caprae)
Oleanber (Nerium oleander)
Searlet Pimpernel (Anagallis arvensis)
LAntana (Lantana camara)
Verbascum (Verbascum snuatuu)
Common Mallow (Malva sylvestris)
Spanish Oyster (Scolymus hispanicum)
Storle's bill (Erodium malacoides)
Calla Lily (Zantedeschia aethiopica)
Blue Gem (Hebes x franciscana)
Bindweed (Convolvulus arvensis)
विभिन्न परिवारों के फूलों के 

In [0]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
hindi_words_counter = collections.Counter([word for sentence in hindi_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} Hindi words.'.format(len([word for sentence in hindi_sentences for word in sentence.split()])))
print('{} unique Hindi words.'.format(len(hindi_words_counter)))
print('10 Most common words in the Hindi dataset:')
print('"' + '" "'.join(list(zip(*hindi_words_counter.most_common(10)))[0]) + '"')

3484 English words.
1855 unique English words.
10 Most common words in the English dataset:
"the" "of" "in" "a" "and" "india" "on" "at" "with" "by"

3762 Hindi words.
2010 unique Hindi words.
10 Most common words in the Hindi dataset:
"में" "के" "एक" "की" "का" "," "पर" "और" "भारत" "है।"


In [0]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [0]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    return pad_sequences(x, maxlen=length, padding='post')

test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


In [0]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_hindi_sentences, english_tokenizer, hindi_tokenizer =\
    preprocess(english_sentences, hindi_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_hindi_sequence_length = preproc_hindi_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
hindi_vocab_size = len(hindi_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max Hindi sentence length:", max_hindi_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("Hindi vocabulary size:", hindi_vocab_size)

Data Preprocessed
Max English sentence length: 146
Max Hindi sentence length: 154
English vocabulary size: 1738
Hindi vocabulary size: 1906


In [0]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [0]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant


BASE_DIR = '/content/drive/My Drive/'
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))


print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(text_tokenizer.word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in text_tokenizer.word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed


Indexing word vectors.
Found 400000 word vectors.
Preparing embedding matrix.


In [0]:
from keras.callbacks import ModelCheckpoint
def embed_model(input_shape, output_sequence_length, english_vocab_size, hindi_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param hindi_vocab_size: Number of unique Hindi words in the dataset
    :return: Keras model built, but not trained
    """
    # Hyperparameters
    learning_rate = 0.005
    
    # TODO: Build the layers
    model = Sequential()
    embedding_layer = Embedding(english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:],
                            embeddings_initializer=Constant(embedding_matrix),
                            trainable=True)
    model.add(embedding_layer)
    model.add(GRU(256, return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(hindi_vocab_size+1, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_hindi_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_hindi_sentences.shape[-2]))

# TODO: Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_hindi_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(hindi_tokenizer.word_index)+1)
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
embed_rnn_model.summary()

embed_rnn_model.fit(tmp_x, preproc_hindi_sentences, batch_size=50, epochs=1500, validation_split=0.01, callbacks=callbacks_list)

# TODO: Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], hindi_tokenizer))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 154, 256)          445184    
_________________________________________________________________
gru_1 (GRU)                  (None, 154, 256)          393984    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 154, 1024)         263168    
_________________________________________________________________
dropout_1 (Dropout)          (None, 154, 1024)         0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 154, 1908)         1955700   
Total params: 3,058,036
Trainable params: 3,058,036
Non-trainable params: 0
_______________________________________________________

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:707: RuntimeWarning: Can save best model only with val_accuracy available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


247/247 [==============================] - 15s 62ms/step - loss: 0.7897 - acc: 0.9028 - val_loss: 0.7747 - val_acc: 0.9048
Epoch 3/1500
247/247 [==============================] - 15s 62ms/step - loss: 0.7479 - acc: 0.8999 - val_loss: 0.7632 - val_acc: 0.9048
Epoch 4/1500
247/247 [==============================] - 15s 62ms/step - loss: 0.7112 - acc: 0.9036 - val_loss: 0.7542 - val_acc: 0.9026
Epoch 5/1500
247/247 [==============================] - 15s 62ms/step - loss: 0.6814 - acc: 0.9044 - val_loss: 0.7476 - val_acc: 0.9026
Epoch 6/1500
247/247 [==============================] - 15s 62ms/step - loss: 0.6716 - acc: 0.9052 - val_loss: 0.8316 - val_acc: 0.9026
Epoch 7/1500
247/247 [==============================] - 15s 62ms/step - loss: 0.6513 - acc: 0.9055 - val_loss: 0.7884 - val_acc: 0.9026
Epoch 8/1500
247/247 [==============================] - 15s 62ms/step - loss: 0.6171 - acc: 0.9052 - val_loss: 0.8234 - val_acc: 0.9026
Epoch 9/1500
247/247 [==============================] - 15s 6

In [0]:
# Print prediction(s)
print("Prediction:")
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], hindi_tokenizer))

print("\nCorrect Translation:")
print(hindi_sentences[0].replace('\n', ' '))

print("\nOriginal text:")
print(english_sentences[0])



Prediction:
st bernarp's lily anthericum liliago bermuda buttereup oxalis pes caprae oleanber nerium oleander searlet pimpernel anagallis arvensis lantana lantana camara verbascum verbascum snuatuu common mallow malva sylvestris spanish oyster scolymus hispanicum storle's bill erodium malacoides calla lily zantedeschia aethiopica blue gem hebes x franciscana bindweed convolvulus <PAD> विभिन्न परिवारों के फूलों के पौधों की बारह प्रजातियों द्वारा उत्पादित फूलों या फूलों के गुच्छों के साथ एक पोस्टर। <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 

In [0]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, hindi_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    
    # Hyperparameters
    learning_rate = 0.001
    
    # Build the layers    
    model = Sequential()
    # Encoder
    model.add(GRU(256, input_shape=input_shape[1:], go_backwards=True))
    model.add(RepeatVector(output_sequence_length))
    # Decoder
   # input_shape = input_shape.reshape((-1, preproc_hindi_sentences.shape[-1]))
    model.add(GRU(256, return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(hindi_vocab_size+1, activation='softmax')))

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

# Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_hindi_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_hindi_sentences.shape[-2],1))

# Train and Print prediction(s)
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    preproc_hindi_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(hindi_tokenizer.word_index)+1)

encdec_rnn_model.summary()

encdec_rnn_model.fit(tmp_x, preproc_hindi_sentences, batch_size=1024, epochs=500, validation_split=0.2)


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_2 (GRU)                  (None, 256)               198144    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 154, 256)          0         
_________________________________________________________________
gru_3 (GRU)                  (None, 154, 256)          393984    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 154, 1024)         263168    
_________________________________________________________________
dropout_2 (Dropout)          (None, 154, 1024)         0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 154, 1908)         1955700   
Total params: 2,810,996
Trainable params: 2,810,996
Non-trainable params: 0
____________________________________________

KeyboardInterrupt: ignored

In [0]:


print("Prediction:")
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], hindi_tokenizer))

print("\nCorrect Translation:")
print(hindi_sentences[0].replace('\n', ' '))

print("\nOriginal text:")
print(english_sentences[0])

Prediction:
मोहनदास करमचन्द गांधी २ अक्टूबर १८६९ पोरबंदर काठियावाड़ गुजरात जन्म ३० जनवरी जनवरी १९४८ वर्ष वर्ष आयु आयु नई नई दिल्ली <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
मोहनदास करमचन्द गांधी २ अक्टूबर १८६९ पोरबंदर, काठियावाड़, गुजरात, जन्म ३० जनवरी १९४८ 

In [0]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
dt = pd.read_csv('/content/drive/My Drive/dataSet4.csv')
dt.insert(5,'input_and_context','EMPTY')
for index, row in dt.iterrows():
  row['input_and_context'] = str(row['english']).lower()
  #row['input_and_context'] = str(row['english']).lower() + ' <START> ' + str(row['context']).lower()
english_sentences_t = dt['input_and_context']
hindi_sentences_t = dt['hindi']
print('Dataset Loaded')
    
preproc_english_sentences_t, preproc_hindi_sentences_t, english_tokenizer_t, hindi_tokenizer_t =\
    preprocess(english_sentences, hindi_sentences)


tmp_x_t = pad(preproc_english_sentences_t, preproc_hindi_sentences_t.shape[1])
tmp_x_t = tmp_x.reshape((-1, preproc_hindi_sentences_t.shape[-2]))

# Print prediction(s)

for i in range(hindi_sentences_t.shape[0]):
  print("\n\nPrediction:")
  print(logits_to_text(embed_rnn_model.predict(tmp_x_t[i:i+1])[0], hindi_tokenizer_t))
  
  print("\nCorrect Translation:")
  print(hindi_sentences_t[i].replace('\n', ' '))
  
  print("\nOriginal text:")
  print(english_sentences_t[i])

Mounted at /content/drive
Dataset Loaded


Prediction:
st bernarp's lily anthericum liliago bermuda buttereup oxalis pes caprae oleanber nerium oleander searlet pimpernel anagallis arvensis lantana lantana camara verbascum verbascum snuatuu common mallow malva sylvestris spanish oyster scolymus hispanicum storle's bill erodium malacoides calla lily zantedeschia aethiopica blue gem hebes x franciscana bindweed convolvulus <PAD> विभिन्न परिवारों के फूलों के पौधों की बारह प्रजातियों द्वारा उत्पादित फूलों या फूलों के गुच्छों के साथ एक पोस्टर। <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

In [0]:
import tensorflow as tf
print(tf.__version__)

1.15.0
